<a href="https://colab.research.google.com/github/hruthiksiva/KaggleComp-IIIT-Delhi/blob/main/Bert_Model_Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 4.8MB/s 
     |████████████████████████████████| 1.1MB 29.9MB/s 
     |████████████████████████████████| 890kB 38.6MB/s 
     |████████████████████████████████| 2.9MB 42.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5156b13d41e473d607a16bfb0e9ca818106bbf8bec92a2b7a3a0b3f016253e14
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# New Section

In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('trainpre.csv')
#dft=pd.read_csv("dft_nolem.csv")
# Select required columns
data = data[['content','type']]
# Remove a row if any of the three remaining columns are missing
data = data.dropna()
enc=LabelEncoder()
data.type = enc.fit_transform(data.type)
# Set your model output as categorical and save in new label col
data['Issue_label'] = pd.Categorical(data['type'])
# Transform your output to numeric
data['Issue'] = data['Issue_label'].cat.codes
# Split into train and test - stratify over Issue

In [ ]:
dft=pd.read_csv("trainpre.csv")
dft.content=dft.content.apply(lambda x:'neutral' if type(x)!=str else x)

In [ ]:
# Name of the BERT model to use
model_name = 'bert-base-uncased'
# Max length of tokens
max_length = 28
# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# TF Keras documentation: https://www.tensorflow.org/api_docs/python/tf/keras/Model
# Load the MainLayer
bert = transformer_model.layers[0]
# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)
# Then build your model output
issue = Dense(units=len(data.Issue_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='issue')(pooled_output)
#product = Dense(units=len(data.Product_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='product')(pooled_output)
outputs = {'issue': issue}
# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')
# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 28)]              0         
_________________________________________________________________
bert (TFBertMainLayer)       ((None, 28, 768), (None,  109482240 
_________________________________________________________________
pooled_output (Dropout)      (None, 768)               0         
_________________________________________________________________
issue (Dense)                (None, 13)                9997      
Total params: 109,492,237
Trainable params: 109,492,237
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss = {'issue': CategoricalCrossentropy(from_logits = True)}
metric = {'issue': CategoricalAccuracy('accuracy')}
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)
# Ready output data for the model
y_issue = to_categorical(data['Issue'])
# Tokenize the input (takes some time)
x = tokenizer(
    text=data['content'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'issue': y_issue},
    batch_size=64,
    epochs=2)

Epoch 1/2
468/468 [==============================] - 179s 383ms/step - loss: 1.8127 - accuracy: 0.3927
Epoch 2/2
468/468 [==============================] - 189s 403ms/step - loss: 1.6639 - accuracy: 0.4476


In [ ]:
dft.content=dft.content.apply(lambda x:'neutral' if type(x)!=str else x)

In [ ]:
#######################################
### ----- Evaluate the model ------ ###
# Ready test data
#test_y_issue = to_categorical(data_test['Issue'])
test_x = tokenizer(
    text=dft['content'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False)


In [ ]:
pred=model.predict(x={'input_ids': test_x['input_ids']})

In [ ]:
pred

{'issue': array([[-2.2018588 , -1.856821  , -0.31007597, ...,  0.91233295,
          1.6640259 ,  1.9119693 ],
        [-2.1690643 , -1.424999  ,  0.44186297, ...,  0.5081822 ,
          0.7094802 ,  1.3417643 ],
        [-2.4448822 , -2.508407  , -0.85149425, ..., -0.02990201,
          0.6318942 ,  1.3429893 ],
        ...,
        [-2.491708  , -2.7157886 , -1.6532431 , ...,  0.5068197 ,
          0.28440487,  1.028627  ],
        [-2.0534747 , -2.4420645 , -1.5925593 , ..., -0.18540414,
          0.3044669 ,  0.45171693],
        [-1.1996839 , -0.9397103 , -0.11683921, ...,  1.257096  ,
          0.7531967 ,  1.9851556 ]], dtype=float32)}

In [ ]:
c=pd.Series(pred["issue"].argmax(axis=1))

In [ ]:
c.value_counts()

12    11063
8      6860
5      6391
7      2648
10     2607
6       425
4         5
11        1
dtype: int64

In [ ]:
c=enc.inverse_transform(c)

In [ ]:
mapper={0:'anger',1: 'boredom', 2:'empty', 3:'enthusiasm', 4:'fun', 5:'happiness', 6:'hate', 7:'love', 8:'neutral', 9:'relief', 10:'sadness', 11:'surprise', 12:'worry'}

In [ ]:
c=c.map(mapper)

In [ ]:
c.shape

(30000,)

In [ ]:
output = pd.DataFrame({'id':dft['id'],'type':c})
output.to_csv('termiii.csv',index = False)

In [ ]:
# 3 Epochs,max length=20